# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Dual model with Clarabel attached

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.5.1  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       ga

The lower bound found is 3

In [7]:
solution_summary(model)

* Solver : Dual model with Clarabel attached

* Status
  Result count       : 1
  Termination status : ALMOST_OPTIMAL
  Message from the solver:
  "ALMOST_SOLVED"

* Candidate solution (result #1)
  Primal status      : NEARLY_FEASIBLE_POINT
  Dual status        : NEARLY_FEASIBLE_POINT
  Objective value    : 3.00000e+00
  Dual objective value : 3.00000e+00

* Work counters
  Solve time (sec)   : 4.05476e-01
  Barrier iterations : 19


The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [L09, (6.15)].

[L09] Laurent, Monique.
*Sums of squares, moment matrices and optimization over polynomials.*
Emerging applications of algebraic geometry (2009): 157-270.

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  1.0000000037842636     -1.0000047782911436     …   1.9058344396239944e-7
 -1.0000047782911436      1.000009571383643          2.541532255451031e-7
 -3.1550056195145586e-6   3.167603764781082e-6       6.86063758087209e-7
  1.0000095848169062     -1.0000143756218862         5.227540250858749e-5
  3.1547525734022624e-6  -3.1479695710497564e-6      5.2563967793825295e-5
  4.007365425383158e-8    8.230483587696018e-12  …   0.00010545597310381817
 -1.0000142970038506      1.00001918446773          -1.3533461258261501e-5
 -3.097903520871693e-6    3.092487890201229e-6       0.0007251441986790519
 -8.48239367762251e-8     2.319938184454498e-7       0.00047263045033047065
  9.699197146534966e-8    2.24157637016163e-8        0.001482860877659785
  1.000019231851

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = measure(ν, atol = 1e-5)

Measure{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}([1.0000000037842636, -1.0000047782911436, -3.1550056195145586e-6, 1.0000095848169062, 3.1547525734022624e-6, 4.007365425383158e-8, -1.0000142970038506, -3.097903520871693e-6, -8.48239367762251e-8, 9.699197146534966e-8  …  0.001482860877659785, 1.342374430106313, -0.25826238957136527, 0.384442405563846, -0.19494757233297494, 0.47961501571279713, -0.052027943282962216, 0.6946617317078664, 0.27099926863682305, 1.1808158676568399], DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³  …  x₁⁷, x₂⁸,

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  1.0000000037842636     -1.0000047782911436     …  9.699197146534966e-8
 -1.0000047782911436      1.0000095848169062        1.4563107571374701e-8
 -3.1550056195145586e-6   3.1547525734022624e-6     1.9058344396239944e-7
  1.0000095848169062     -1.0000142970038506        9.456624269134494e-8
  3.1547525734022624e-6  -3.097903520871693e-6      2.541532255451031e-7
  4.007365425383158e-8   -8.48239367762251e-8    …  6.86063758087209e-7
 -1.0000142970038506      1.0000192318510108        1.745477371543284e-5
 -3.097903520871693e-6    3.105396502408875e-6      5.227540250858749e-5
 -8.48239367762251e-8     3.1818786158592063e-7     5.2563967793825295e-5
  9.699197146534966e-8    1.4563107571374701e-8     0.00010545597310381817

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 4.00006203889537
 0.00016394383941014867
 3.655817239577864e-5
 5.244097495812441e-7
 3.9503234805812473e-7
 2.3507116219028302e-7
 1.8680049474206079e-7
 3.6952757694002633e-8
 3.416466221175507e-8
 1.2004294306188961e-8

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.103068949030815
 1.8099461582903524
 1.1685546652319592
 0.00041578036099193993
 0.00012878184022232887
 3.0373817974177292e-5
 2.731833879947775e-5
 4.6795295038040007e-7
 3.749071933639144e-8
 2.205668317259579e-8
 4.4875761955384996e-9
 2.4721470127377835e-9
 5.555260481320318e-10
 3.5532318311039704e-11
 1.0716401015848837e-11

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [-3.0872956816092415e-6, -1.0000047513625485] with weight 1.0228434412609957

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*